### Impute masked UKBB tabular data

In [1]:
import os
from os.path import join
import random
import pandas as pd
import numpy as np
import torch

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer


/home/siyi/miniconda3/envs/selfsuper/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
DVM_column_names = [ 'Color', 'Bodytype', 'Gearbox','Fuel type' ,
               'Wheelbase', 'Height', 'Width', 'Length', 'Adv year', 'Adv month', 'Reg year', 'Runned miles', 'Price', 'Seat num', 'Door num', 'Entry price', 'Engine size',]
CAD_column_names = torch.load('/bigdata/siyi/data/UKBB/cardiac_segmentations/projects/SelfSuperBio/18545/cardiac_column_names_reordered.pt')
BASE = '/bigdata/siyi/data'
DATA_TRAIN = join(BASE, 'UKBB/cardiac_segmentations/projects/SelfSuperBio/18545/final/cardiac_features_train_imputed_noOH_tabular_imaging_reordered.csv')
DATA_TEST = join(BASE, 'UKBB/cardiac_segmentations/projects/SelfSuperBio/18545/final/cardiac_features_test_imputed_noOH_tabular_imaging_reordered.csv')
ratio = 0.1
DATA_MASK = join(BASE, f'UKBB/cardiac_segmentations/projects/SelfSuperBio/18545/final/missing_mask/cardiac_features_test_imputed_noOH_tabular_imaging_reordered_CAD_value_{ratio}.npy')

In [62]:
df_train = pd.read_csv(DATA_TRAIN, header=None)
df_test = pd.read_csv(DATA_TEST, header=None)
mask = np.load(DATA_MASK)
df_test

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.373658,-0.906652,-0.204426,1.892559,-0.600613,-1.103572,-0.862449,-1.223299,-0.298868,1.489236
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-1.094133,-0.353167,0.485431,1.886483,0.427886,-0.565698,-0.219816,-0.555483,0.177944,0.820836
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.598308,-1.979092,-1.861468,0.679597,-1.511450,-1.342627,-1.612787,-0.964029,-1.939037,-0.959561
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.464696,1.032027,1.343418,0.395740,0.016951,0.697206,1.313170,0.751864,1.613297,0.325992
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.953512,-0.776421,-0.400290,0.973278,-1.648825,-0.128478,-0.355344,-0.187792,-0.452890,-0.260055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.198964,-0.496224,-0.670686,-0.436369,-0.280841,-0.356524,-0.380474,-0.489487,-0.184171,0.412087
3613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.145579,-0.901720,-0.426290,1.287247,-0.953581,-1.344200,-0.855270,-1.110164,-0.403734,1.193079
3614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.140753,-0.330729,-0.438773,-0.284430,0.768589,-0.273151,-0.892432,-0.467440,-1.141788,-0.636715
3615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.459626,0.131256,0.695160,1.076148,-0.075851,-0.638044,0.542189,0.234896,0.744875,0.265203


In [63]:
mask.shape

(3617, 75)

In [64]:
# Mask the test 
df_test_masked = df_test.copy()
df_test_masked.values[mask] = np.nan
df_test_masked

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,-1.373658,-0.906652,NaN,1.892559,NaN,-1.103572,-0.862449,NaN,-0.298868,1.489236
1,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-1.094133,-0.353167,0.485431,1.886483,0.427886,-0.565698,-0.219816,-0.555483,0.177944,0.820836
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.598308,-1.979092,-1.861468,0.679597,-1.511450,NaN,-1.612787,-0.964029,-1.939037,-0.959561
3,0.0,1.0,0.0,NaN,NaN,0.0,1.0,NaN,0.0,0.0,...,0.464696,1.032027,1.343418,0.395740,0.016951,0.697206,1.313170,0.751864,1.613297,0.325992
4,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,-0.953512,-0.776421,-0.400290,0.973278,-1.648825,-0.128478,-0.355344,-0.187792,-0.452890,-0.260055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,-0.198964,-0.496224,-0.670686,-0.436369,-0.280841,-0.356524,-0.380474,-0.489487,-0.184171,0.412087
3613,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.145579,-0.901720,-0.426290,1.287247,-0.953581,-1.344200,NaN,-1.110164,-0.403734,1.193079
3614,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,...,-0.140753,-0.330729,-0.438773,-0.284430,0.768589,-0.273151,-0.892432,-0.467440,-1.141788,-0.636715
3615,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,-0.459626,0.131256,0.695160,1.076148,NaN,-0.638044,0.542189,0.234896,0.744875,0.265203


In [65]:
N_test = df_test.shape[0]
N_train = df_train.shape[0]
df_all = pd.concat([df_train, df_test_masked], axis=0).reset_index(drop=True)

In [66]:
num_cat = 26
cat_cols = df_all.columns[:num_cat].tolist()
cont_cols = df_all.columns[num_cat:].tolist()

cat_imputer = IterativeImputer(
    estimator=RandomForestClassifier(),
    max_iter=20,
    random_state=42,
    initial_strategy="most_frequent",
    skip_complete=False
)
cont_imputer = IterativeImputer(
    max_iter=20,
    random_state=42
)

preprocessor = ColumnTransformer([
    ("cat", cat_imputer, cat_cols),
    ("cont", cont_imputer, cont_cols)
])

df_all_filled = preprocessor.fit_transform(df_all)
df_all_filled = pd.DataFrame(df_all_filled, columns=df_all.columns)

/home/siyi/miniconda3/envs/selfsuper/lib/python3.9/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/home/siyi/miniconda3/envs/selfsuper/lib/python3.9/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [67]:
df_test_filled = df_all_filled.iloc[N_train:, :].reset_index(drop=True)
df_test_filled

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.373658,-0.906652,-0.204426,1.892559,-0.616939,-1.103572,-0.862449,-1.223299,-0.298868,1.489236
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,-1.094133,-0.353167,0.485431,1.886483,0.427886,-0.565698,-0.219816,-0.555483,0.177944,0.820836
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.598308,-1.979092,-1.861468,0.679597,-1.511450,-1.368904,-1.612787,-0.964029,-1.939037,-0.959561
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.464696,1.032027,1.343418,0.395740,0.016951,0.697206,1.313170,0.751864,1.613297,0.325992
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.953512,-0.776421,-0.400290,0.973278,-1.648825,-0.128478,-0.355344,-0.187792,-0.452890,-0.260055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.198964,-0.496224,-0.670686,-0.436369,-0.280841,-0.356524,-0.380474,-0.489487,-0.184171,0.412087
3613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.145579,-0.901720,-0.426290,1.287247,-0.953581,-1.344200,-0.855270,-1.110164,-0.403734,1.193079
3614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.140753,-0.330729,-0.438773,-0.284430,0.768589,-0.273151,-0.892432,-0.467440,-1.141788,-0.636715
3615,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.459626,0.131256,0.695160,1.076148,0.167506,-0.638044,0.542189,0.234896,0.744875,0.265203


In [68]:
# Use accuracy for categorical columns and MSE for continuous columns to evaluate imputation performance (only on the masked entries)
from sklearn.metrics import mean_squared_error, accuracy_score
acc_list = []
for col in cat_cols:
    true = df_test.iloc[:, col]
    pred = df_test_filled.iloc[:, col]
    mask_col = mask[:, col]
    acc = accuracy_score(true[mask_col], pred[mask_col])
    acc_list.append(acc)
print("Categorical columns accuracy:", np.mean(acc_list))
mse_list = []
for col in cont_cols:
    true = df_test.iloc[:, col]
    pred = df_test_filled.iloc[:, col]
    mask_col = mask[:, col]
    mse = mean_squared_error(true[mask_col], pred[mask_col])
    mse_list.append(mse)
print("Continuous columns MSE:", np.mean(mse_list))
results = pd.DataFrame({
    "Column": CAD_column_names,
    "MSE": [0] * (len(cat_cols)) + mse_list,
    "Accuracy": acc_list + [0] * (len(cont_cols)),
})
results

Categorical columns accuracy: 0.7672958167488495
Continuous columns MSE: 0.46092564405346703


,Column,MSE,Accuracy
0,Shortness of breath walking on level ground-2.0,0.000000,0.915989
1,Sex-0.0,0.000000,0.635616
2,Diabetes diagnosis,0.000000,0.962406
3,Angina diagnosed by doctor,0.000000,0.972299
4,Stroke diagnosed by doctor,0.000000,0.994334
...,...,...,...
70,LVM (g),0.149137,0.000000
71,RVEDV (mL),0.053664,0.000000
72,RVESV (mL),0.050770,0.000000
73,RVSV (mL),0.125807,0.000000


In [69]:
# Save the imputed test set and results
df_test_filled.to_csv(DATA_TEST[:-4]+f'_mask_{ratio}_filled.csv', index=False, header=False)
results.to_csv(f'CAD_balanced_mask_{ratio}_imputation_results.csv', index=False)